In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, BatchNormalization, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

In [23]:
data = pd.read_csv("./CSV_Files/glucose_data.csv")
if 'Glucose_time' not in data.columns:
    data["Glucose_time"] = pd.to_datetime(data['reading_time'], unit='ms')
    data['Glucose_time'] = pd.to_datetime(data['Glucose_time'], format='%Y-%m-%d %H:%M:%S')
    data = data.drop(columns=['reading_time'])
    data = data.sort_values(by='Glucose_time')

In [24]:
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
data["Glucose_time"] = pd.to_datetime(data["Glucose_time"])

In [25]:
df = data[['Glucose_time', 'reading']]
df.set_index('Glucose_time', inplace=True)
df = df.resample('5min').mean().interpolate(method='linear')
df.head()

,reading
Glucose_time,
2019-08-26 19:10:00,108.1092
2019-08-26 19:15:00,200.5546
2019-08-26 19:20:00,293.0000
2019-08-26 19:25:00,293.0000
2019-08-26 19:30:00,296.0000


In [26]:
df.to_csv('./CSV_Files/glucose_data_resampled.csv')

In [27]:
df = pd.read_csv("./CSV_Files/glucose_data_resampled.csv")
# Drop all the columns which have unnamed in them
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()

,Glucose_time,reading
0,2019-08-26 19:10:00,108.1092
1,2019-08-26 19:15:00,200.5546
2,2019-08-26 19:20:00,293.0000
3,2019-08-26 19:25:00,293.0000
4,2019-08-26 19:30:00,296.0000


In [28]:
# If reading_time is present in the dataset, then convert it to a proper date time format
if 'Glucose_time' not in df.columns:
    df['reading_time'] = pd.to_datetime(df['reading_time'], unit='ms')
    df['reading_time'] = pd.to_datetime(df['reading_time'], format='%Y-%m-%d %H:%M:%S')
    df = df.rename(columns={'reading_time': 'Glucose_time'})
    df = df.sort_values(by='Glucose_time')
    df.head()

In [29]:
# Set the Glucose_time to datetime format and set it as the index
df['Glucose_time'] = pd.to_datetime(df['Glucose_time'])
df.set_index('Glucose_time', inplace=True)
df.head()

,reading
Glucose_time,
2019-08-26 19:10:00,108.1092
2019-08-26 19:15:00,200.5546
2019-08-26 19:20:00,293.0000
2019-08-26 19:25:00,293.0000
2019-08-26 19:30:00,296.0000


In [30]:
checked_df = df.copy()

In [31]:
scaler = MinMaxScaler(feature_range=(0, 1))
df['reading'] = scaler.fit_transform(df[['reading']])
df.head()

,reading
Glucose_time,
2019-08-26 19:10:00,0.146562
2019-08-26 19:15:00,0.484365
2019-08-26 19:20:00,0.822168
2019-08-26 19:25:00,0.822168
2019-08-26 19:30:00,0.833130


In [32]:
df.shape

(1498, 1)

In [33]:
def prepare_data(series, n_steps):
    X, y = [], []
    for i in range(len(series)):
        end_ix = i + n_steps
        if end_ix > len(series)-1:
            break
        seq_x, seq_y = series[i:end_ix], series[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [34]:
time_series_data = df['reading'].values

In [35]:
n_feat = 1   # Number of features
n_steps = 1  # Further increased number of time steps in each input sample
batch_size_lst = [16, 32, 64]  # Adjusted batch size
patience_lst = [20, 30, 50]  # Adjusted patience
best_rmse = float('inf')
best_parameter = {}
best_predictions = None
best_model = None

In [36]:
def lr_schedule(epoch):
    if epoch < 10:
        return 0.001
    else:
        return 0.001 * (0.995 ** (epoch - 9))

In [37]:
X_train, y_train = prepare_data(time_series_data, n_feat)
X_train.shape

(1497, 1)

In [38]:
for batch_size in batch_size_lst:
    for patience in patience_lst:

        train_data_scaled = time_series_data[:-20]
        test_data_scaled = time_series_data[-(n_steps + 20):]

        X_train, y_train = prepare_data(train_data_scaled, n_steps)
        X_test, y_test = prepare_data(test_data_scaled, n_steps)

        X_train = X_train.reshape(X_train.shape[0], n_steps, 1)
        X_test = X_test.reshape(X_test.shape[0], n_steps, 1)

        # Define the model
        model = Sequential()
        model.add(Conv1D(filters=128, kernel_size=1, activation='relu', input_shape=(n_steps, 1), kernel_regularizer=l2(0.01)))
        model.add(MaxPooling1D(pool_size=1))
        model.add(Dropout(0.2))


        model.add(LSTM(units=200, activation='tanh', return_sequences=True))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())

        model.add(Bidirectional(LSTM(units=150, activation='relu', return_sequences=True)))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())

        model.add(Bidirectional(LSTM(units=100, activation='relu', return_sequences=True)))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())

        model.add(LSTM(units=50, activation='relu', return_sequences=False))  # Set return_sequences to False
        model.add(Dropout(0.2))
        model.add(BatchNormalization())

        model.add(Dense(100, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(1))

        optimizer = Adam(learning_rate=0.001)
        model.compile(optimizer=optimizer, loss='mse')
        early_stop = EarlyStopping(monitor='val_loss', patience=patience, verbose=1, restore_best_weights=True)
        lr_scheduler = LearningRateScheduler(lr_schedule)

        # Train the model
        history = model.fit(X_train, y_train, epochs=500, batch_size=batch_size, verbose=1, validation_split=0.2, callbacks=[early_stop, lr_scheduler])
        # Make predictions
        predictions = []
        curr_sequence = X_test[0].reshape(1, n_steps, 1)
        for i in range(20):
            next_pred = model.predict(curr_sequence)
            predictions.append(next_pred[0, 0])
            curr_sequence = np.append(curr_sequence[:, 1:, :], next_pred.reshape(1, 1, 1), axis=1)

        # Inverse transform the predictions
        predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
        actual = checked_df['reading'].values[-20:]

        # Calculate RMSE
        rmse = np.sqrt(mean_squared_error(actual, predictions))
        print(f"RMSE: {rmse}")

        # Update best parameters if current model is better
        if rmse < best_rmse:
            best_rmse = rmse
            best_parameter = {'n_steps': n_steps, 'dropout_rate': 0.3, 'batch_size': batch_size, 'patience': patience}
            best_predictions = predictions
            best_model = model

        print(f"Best parameters: {best_parameter}")
        print(f"Best RMSE: {best_rmse}")

c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500


74/74 ━━━━━━━━━━━━━━━━━━━━ 30s 50ms/step - loss: 0.2745 - val_loss: 0.1111 - learning_rate: 0.0010
Epoch 2/500
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0515 - val_loss: 0.0818 - learning_rate: 0.0010
Epoch 3/500
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0382 - val_loss: 0.0658 - learning_rate: 0.0010
Epoch 4/500
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0333 - val_loss: 0.0591 - learning_rate: 0.0010
Epoch 5/500
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0303 - val_loss: 0.0556 - learning_rate: 0.0010
Epoch 6/500
74/74 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0244 - val_loss: 0.0528 - learning_rate: 0.0010
Epoch 7/500
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - loss: 0.0234 - val_loss: 0.0440 - learning_rate: 0.0010
Epoch 8/500
74/74 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 0.0212 - val_loss: 0.0389 - learning_rate: 0.0010
Epoch 9/500
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0180 - val_loss: 0.0263 - learning_rate: 0.0010
Epoch 10/500
74/74 ━━

c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500
74/74 ━━━━━━━━━━━━━━━━━━━━ 51s 64ms/step - loss: 0.1746 - val_loss: 0.1267 - learning_rate: 0.0010
Epoch 2/500
74/74 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0582 - val_loss: 0.0966 - learning_rate: 0.0010
Epoch 3/500
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - loss: 0.0436 - val_loss: 0.0730 - learning_rate: 0.0010
Epoch 4/500
74/74 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.0360 - val_loss: 0.0632 - learning_rate: 0.0010
Epoch 5/500
74/74 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0280 - val_loss: 0.0617 - learning_rate: 0.0010
Epoch 6/500
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0273 - val_loss: 0.0601 - learning_rate: 0.0010
Epoch 7/500
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0238 - val_loss: 0.0484 - learning_rate: 0.0010
Epoch 8/500
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0218 - val_loss: 0.0378 - learning_rate: 0.0010
Epoch 9/500
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0189 - val_loss: 0.0337 - learning_rate: 0.0010
Epoch 10/

...

Epoch 1/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 29s 134ms/step - loss: 0.8987 - val_loss: 0.1197 - learning_rate: 0.0010
Epoch 2/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.1276 - val_loss: 0.1297 - learning_rate: 0.0010
Epoch 3/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0827 - val_loss: 0.1182 - learning_rate: 0.0010
Epoch 4/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0582 - val_loss: 0.1044 - learning_rate: 0.0010
Epoch 5/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0539 - val_loss: 0.0954 - learning_rate: 0.0010
Epoch 6/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0481 - val_loss: 0.0859 - learning_rate: 0.0010
Epoch 7/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0520 - val_loss: 0.0775 - learning_rate: 0.0010
Epoch 8/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0427 - val_loss: 0.0718 - learning_rate: 0.0010
Epoch 9/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0382 - val_loss: 0.0675 - learning_rate: 0.0010
Epoch 10

c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 44s 230ms/step - loss: 1.3752 - val_loss: 0.1119 - learning_rate: 0.0010
Epoch 2/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - loss: 0.1789 - val_loss: 0.1190 - learning_rate: 0.0010
Epoch 3/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0816 - val_loss: 0.0999 - learning_rate: 0.0010
Epoch 4/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 0.0748 - val_loss: 0.0908 - learning_rate: 0.0010
Epoch 5/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0592 - val_loss: 0.0814 - learning_rate: 0.0010
Epoch 6/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0488 - val_loss: 0.0731 - learning_rate: 0.0010
Epoch 7/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0489 - val_loss: 0.0679 - learning_rate: 0.0010
Epoch 8/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0433 - val_loss: 0.0631 - learning_rate: 0.0010
Epoch 9/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0352 - val_loss: 0.0614 - learning_rate: 0.0010
Epoch 10

c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 31s 202ms/step - loss: 0.1932 - val_loss: 0.1374 - learning_rate: 0.0010
Epoch 2/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0769 - val_loss: 0.1195 - learning_rate: 0.0010
Epoch 3/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0625 - val_loss: 0.1035 - learning_rate: 0.0010
Epoch 4/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0571 - val_loss: 0.0885 - learning_rate: 0.0010
Epoch 5/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0420 - val_loss: 0.0789 - learning_rate: 0.0010
Epoch 6/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.0382 - val_loss: 0.0706 - learning_rate: 0.0010
Epoch 7/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0329 - val_loss: 0.0652 - learning_rate: 0.0010
Epoch 8/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 0.0294 - val_loss: 0.0633 - learning_rate: 0.0010
Epoch 9/500
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0293 - val_loss: 0.0611 - learning_rate: 0.0010
Epoch 10

In [39]:
df_test = pd.DataFrame({'actual': actual, 'predictions': best_predictions})
df_test

,actual,predictions
0,333.444444,300.864777
1,337.555556,285.110474
2,341.666667,274.246948
3,337.666667,267.005310
4,336.000000,262.402466
5,330.000000,259.542023
6,324.000000,257.852570
7,318.000000,256.866669
8,312.500000,256.300842
9,307.000000,255.980499


In [44]:
initial_value = 208
scaled_initial_value = scaler.transform([[initial_value]])

# Create an initial sequence with the scaled initial value repeated to match n_steps
initial_sequence = np.array([scaled_initial_value for _ in range(n_steps)]).reshape(1, n_steps, 1)

# Generate predictions
predictions = []
current_sequence = initial_sequence
for i in range(10):
    next_pred = model.predict(current_sequence)
    predictions.append(next_pred[0, 0])
    current_sequence = np.append(current_sequence[:, 1:, :], next_pred.reshape(1, 1, 1), axis=1)

# Inverse transform the predictions
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()

print("Next 10 predictions:", predictions)

c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Next 10 predictions: [206.16246 204.55353 203.20651 202.21858 201.59425 201.14532 200.81384
 200.594   200.45772 200.36589]
